In [2]:
import dill
import pandas as pd
import numpy as np
import requests
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, MaxAbsScaler, FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor

from keras.applications.vgg16 import VGG16, preprocess_input
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, concatenate, Flatten, Dropout 
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin
import os


### Extract Image Features using pretrained CNN Model

In [ ]:
image_size = (224, 224)

# base model for transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

%%time
image_size = (224, 224)
zoom_level=[16.25, 11.5]
parent_folder=r'.\Capstone\Satellite Images'
satellite_images=[]

subfolder0=os.path.join(parent_folder, f'zoom-{zoom_level[0]}')
subfolder1=os.path.join(parent_folder, f'zoom-{zoom_level[1]}')
counter=0
batch_size=5000

images_array0 = np.empty((0, image_size[0], image_size[1], 3), dtype=np.float32)
images_array1 = np.empty((0, image_size[0], image_size[1], 3), dtype=np.float32)

image_features = np.empty((0,50176), dtype=np.float32)


for Id in X_final['id'][:10]:  
    
    file_name0 = Id+f'-{zoom_level[0]}.jpg'
    file_name1 = Id+f'-{zoom_level[1]}.jpg'
    img_path0 = os.path.join(subfolder0, file_name0)
    img_path1 = os.path.join(subfolder1, file_name1)
    
    img0=load_img(img_path0, target_size=image_size) #load and process image at zoom_level[0]
    img0 = img_to_array(img0)
    img0 = preprocess_input(img0)
    img0= np.expand_dims(img0, axis=0) #add extra dimension for appending
    images_array0=np.append(images_array0, img0, axis=0)
    
    img1=load_img(img_path1, target_size=image_size) #load and process image at zoom_level[1]
    img1 = img_to_array(img1)
    img1 = preprocess_input(img1)
    img1=np.expand_dims(img1, axis=0) #add extra dimension for appending
    images_array1=np.append(images_array1, img1, axis=0)



    if counter%batch_size==0:
        
        img_features0 = base_model.predict(images_array0, verbose=0)
        img_features0 = img_features0.reshape(img_features0.shape[0], -1) #flatten feature matrices 
                
        img_features1 = base_model.predict(images_array1, verbose=0)
        img_features1 = img_features1.reshape(img_features1.shape[0], -1) #flatten feature matrices 
        
        feature_stack = np.hstack((img_features0, img_features1))
        image_features= np.append(image_features, feature_stack, axis=0)
        
        #clear image arrays for next batch        
        images_array0 = np.empty((0, image_size[0], image_size[1], 3), dtype=np.float32)
        images_array1 = np.empty((0, image_size[0], image_size[1], 3), dtype=np.float32)
   
       
    counter+=1
    if counter%5000==0:
        print(counter,'images processed')
        
#process remaining images if any
img_features0 = base_model.predict(images_array0, verbose=0)
img_features0 = img_features0.reshape(img_features0.shape[0], -1) #flatten feature matrices 
img_features1 = base_model.predict(images_array1, verbose=0)
img_features1 = img_features1.reshape(img_features1.shape[0], -1) #flatten feature matrices 

feature_stack = np.hstack((img_features0, img_features1))
image_features= np.append(image_features, feature_stack, axis=0)
    

In [ ]:
from sklearn.decomposition import PCA

#reducing demionality of image features data using PCA to make it easier to process

%%time
pca = PCA(n_components=2500)  
reduced_features_2500 = pca.fit_transform(image_features)

# Explained variance ratio
explained_variance = pca.explained_variance_ratio_

#print(f"Explained Variance Ratio of PC1 and PC2: {explained_variance}")

sum(explained_variance)


### Pipeline to process Home Features and Demographics Data

In [4]:
def apply_julian(date_df):
    series = date_df.squeeze()
    return np.array(series.apply(lambda x: x.to_julian_date())).reshape(-1, 1)

class DistanceTransfomer(BaseEstimator, TransformerMixin):
    
    def __init__(self, places):
        self.places = places
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        result = []
        for place in self.places:
            result.append(np.sqrt(((X.values - place)**2).sum(axis=1)).reshape(-1, 1))

In [ ]:

#locations
PHL = (39.87368234690642, -75.24350488095705)
City_Hall = (39.95281182498322, -75.1615591562166)
KOP_mall = (40.08864933327422, -75.390381745456)

locations = [PHL, City_Hall, KOP_mall]


categorical_columns=['zipCode']
numeric_columns = ['bathrooms', 'bedrooms', 'squareFootage', 'Median household income', 'Median Value of Owner-occupied units ']


pipe_yearBuilt= Pipeline([('impute', SimpleImputer(strategy='most_frequent'))#,
                          #('poly', PolynomialFeatures(degree=1))
                         ])


pipe_lastSaleDate = Pipeline([('to_julian', FunctionTransformer(apply_julian)),
                              ('impute', SimpleImputer(strategy='median'))#,
                              #('poly', PolynomialFeatures(degree=2))
                             ])

pipe_demo_poly = Pipeline([('impute', SimpleImputer(strategy='median'))#,
                           #('poly', PolynomialFeatures(degree=2))
                          ])



ct = ColumnTransformer([('cat', OneHotEncoder(), categorical_columns),
                        ('numeric_impute', SimpleImputer(strategy='median'), numeric_columns),
                        ('distances', DistanceTransfomer(locations), ['latitude', 'longitude']),
                        ('pipe_yearBuilt', pipe_yearBuilt, ['yearBuilt']),
                        ('pipe_lastSaleDate',  pipe_lastSaleDate, ['lastSaleDate']),
                        ('pipe_demo_poly',  pipe_demo_poly, ['Employed Population %', 'Minority Pop % Calc', 'Median age (years)']),
                       ])

ct_home_features_only = ColumnTransformer([('cat', OneHotEncoder(), categorical_columns),
                        ('numeric_impute', SimpleImputer(strategy='median'), numeric_columns[0:3]),
                        ('distances', DistanceTransfomer(locations), ['latitude', 'longitude']),
                        ('pipe_yearBuilt', pipe_yearBuilt, ['yearBuilt']),
                        ('pipe_lastSaleDate',  pipe_lastSaleDate, ['lastSaleDate'])
                       ])



lin_pipe = Pipeline([('ct_transformer', ct),
                 #('poly', PolynomialFeatures(degree=2)),
                 ('scaler', MaxAbsScaler())
                 #('ridge', Ridge(alpha=0))
                ])


In [6]:
#X and y datasets

with open(r'X_final.pkd', 'rb') as f:
     X_final = dill.load(f)

X=X_final.drop('lastSalePrice', axis=1)
y=X_final['lastSalePrice']

#home_features_sparse=lin_pipe.fit_transform(X)

In [ ]:
#combining reduced image features datasets with the output of lin_pipe

from scipy.sparse import hstack

demo_home_data= lin_pipe.fit_transform(X)

combined_data_3000 = hstack((demo_home_data, reduced_features_3000))
combined_data_2000 = hstack((demo_home_data, reduced_features_2000))

In [ ]:
lin_model = Pipeline([('scaler', MaxAbsScaler()),
                       ('ridge', Ridge(alpha=0))])


grid_searcher_2000= GridSearchCV(lin_model, 
                                 {'ridge__alpha': [0, 1, 10]},
                                 n_jobs=2, 
                                 cv=5)

grid_searcher_3000= GridSearchCV(lin_model, 
                                 {'ridge__alpha': [0, 1, 10]},
                                 n_jobs=2, 
                                 cv=5)

In [ ]:
# Grid search with pca n_components = 2000 

from sklearn.utils import shuffle

combined_data_2000_shuffled, y_shuffled = shuffle(combined_data_2000, y, random_state=42)
grid_searcher_2000.fit(combined_data_2000_shuffled, y_shuffled)

print(grid_searcher_2000.best_params_)
print(grid_searcher_2000.score(combined_data_2000_shuffled, y_shuffled))
grid_searcher_2000.cv_results_

In [ ]:
# Grid search with pca n_components = 3000 

combined_data_3000_shuffled, y_shuffled = shuffle(combined_data_3000, y, random_state=42)
grid_searcher_3000.fit(combined_data_3000_shuffled, y_shuffled)
print(grid_searcher_3000.best_params_)
print(grid_searcher_3000.score(combined_data_3000_shuffled, y_shuffled))
grid_searcher_3000.cv_results_

In [ ]:
# Grid search with pca n_components = 3000 

#copied output for grid_searcher_3000
"""
{'ridge__alpha': 0}
0.7473774897676411
{'mean_fit_time': array([248.07357888,  83.96293612,  58.84812312]),
 'std_fit_time': array([15.07273627,  2.63493217,  4.17996321]),
 'mean_score_time': array([2.26447978, 1.91136951, 1.96938534]),
 'std_score_time': array([0.42178446, 0.82405852, 0.98727506]),
 'param_ridge__alpha': masked_array(data=[0, 1, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'ridge__alpha': 0}, {'ridge__alpha': 1}, {'ridge__alpha': 10}],
 'split0_test_score': array([0.71564913, 0.69625264, 0.67562538]),
 'split1_test_score': array([0.71166675, 0.6904854 , 0.66818436]),
 'split2_test_score': array([0.71999322, 0.70245437, 0.6839968 ]),
 'split3_test_score': array([0.70954531, 0.70098276, 0.68574379]),
 'split4_test_score': array([0.69915166, 0.696173  , 0.6977118 ]),
 'mean_test_score': array([0.71120121, 0.69726963, 0.68225243]),
 'std_test_score': array([0.00700334, 0.00421738, 0.00996045]),
 'rank_test_score': array([1, 2, 3])}
""""

In [ ]:
#non linear model

non_lin_model = Pipeline([#('ct_transformer', ct),
                 #('poly', PolynomialFeatures(degree=2)),
                 ('scaler', MaxAbsScaler()),
                 ('rf', RandomForestRegressor(n_estimators=100, max_depth=20, min_samples_leaf=25))
                ])

In [ ]:
#full model fits a linear model and then fits a non linear model on the residuals of the linear model


class FullModel (BaseEstimator, RegressorMixin):
    
    def __init__ (self, lin_model, non_lin_model):
        self.lin_model = lin_model
        self.non_lin_model = non_lin_model
        
    def fit (self, X, y):
        
        """fit linaer model"""
        self.lin_model.fit(X, y)
        residuals = y - self.lin_model.predict(X)
        
        """fit non linear model"""
        self.non_lin_model.fit(X, residuals)
        
        return self

    def predict(self, X):
        
        lin_pred = self.lin_model.predict(X)
        non_lin_pred = self.non_lin_model.predict(X)
        final_pred = lin_pred + non_lin_pred
        return final_pred

In [ ]:
price_estimator = FullModel(lin_model, non_lin_model)
price_estimator.fit(combined_data_2000, y)

In [7]:
with open('final_model.pkd', 'rb') as f:
    price_estimator = dill.load(f)


In [8]:
with open('combined_data_2000.pkd', 'rb') as f:
    combined_data_2000 = dill.load(f)

In [19]:
from sklearn.metrics import mean_absolute_error, r2_score
# Final Model Results

print(f'mean_abs_error: {mean_absolute_error(y, price_estimator.predict(combined_data_2000))}\
, R Squared: {r2_score(y, price_estimator.predict(combined_data_2000))}')

mean_abs_error: 55850.06112617552, R Squared: 0.8354817609602162
